<a href="https://colab.research.google.com/github/chetuchetan1/7th-Sem-ML-LAB/blob/master/candidate_elimination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import csv

In [0]:
def g_0(n):
  return ("?",)*n
def s_0(n):
  return('0',)*n

In [31]:
s_0(5)

('0', '0', '0', '0', '0')

In [32]:
g_0(5)

('?', '?', '?', '?', '?')

In [33]:
def more_general(h1,h2): #hypothesis
  more_general_parts = [] # empty list
  for x,y in zip(h1,h2):
    mg = x == "?" or (x!= "0" and (x == y or y =="0"))#most general
    more_general_parts.append(mg)
  return all(more_general_parts)

l1 = [1,2,3]
l2 = [3,4,5]

list(zip(l1,l2))

[(1, 3), (2, 4), (3, 5)]

In [0]:
#min generaliztions
def fulfills(example, hypothesis):
  return more_general(hypothesis, example)

def min_generalizations(h, x):
  h_new = list(h)
  for i in range(len(h)):
    if not fulfills(x[i:i+1], h[i:i+1]):
      h_new[i] = '?' if h[i] != '0' else x[i]
  return [tuple(h_new)]

In [35]:
min_generalizations(h=('0','0','sunny'),
                   x=('rainy','windy','cloudy'))

[('rainy', 'windy', '?')]